In [ ]:
import pandas as pd
diff_data_midi = pd.read_csv("/Users/jsenf/Desktop/empra_app/diff_data_midi.csv")
diff_data_midi

In [ ]:
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objects as go
import os

# Laden des DataFrames diff_data_midi (ersetzen Sie dies durch Ihre DataFrame-Lade-Methode)
diff_data_midi = pd.read_csv("/Users/jsenf/Desktop/empra_app/diff_data_midi.csv")

# Liste der verfügbaren Bedingungen und sound_conditions
conditions = [
    "faces_left_faces_right",
    "words_left_words_right",
    "faces_left_words_left",
    "faces_right_words_right",
    "faces_right_words_left",
    "faces_left_words_right"
]
sound_conditions = ["normal", "aesth"]

# Erstellen der Dash-App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("MIDI Note Differences with Audio Synchronization", style={"textAlign": "center"}),

    # Dropdown-Menüs
    html.Div([
        dcc.Dropdown(
            id="condition-dropdown",
            options=[{"label": condition, "value": condition} for condition in conditions],
            value=conditions[0],
            clearable=False,
            style={"width": "48%", "display": "inline-block", "marginRight": "2%"}
        ),
        dcc.Dropdown(
            id="sound-condition-dropdown",
            options=[{"label": sc, "value": sc} for sc in sound_conditions],
            value=sound_conditions[0],
            clearable=False,
            style={"width": "48%", "display": "inline-block"}
        )
    ]),

    # Audio Player
    html.Audio(id="audio-player", 
               controls=True, 
               style={"width": "100%", "marginTop": "20px"}
               ),

    # Graph
    dcc.Graph(id="diff-graph", style={"height": "70vh"}),

    # Hidden Interval Component for Animation
    dcc.Interval(id="interval", interval=100, disabled=True)
])

@app.callback(
    [Output("audio-player", "src"),
     Output("diff-graph", "figure")],
    [Input("condition-dropdown", "value"),
     Input("sound-condition-dropdown", "value")]
)
def update_content(selected_condition, selected_sound_condition):
    # Audio-Dateipfad
    audio_path = f"assets/sounds/{selected_condition}_{selected_sound_condition}.wav"

    # Graph erstellen
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=diff_data_midi["time"],
        y=diff_data_midi[selected_condition],
        mode="lines",
        name=selected_condition,
        line=dict(color="purple")
    ))

    fig.update_layout(
        title=f"Absolute Differences for {selected_condition}",
        xaxis_title="Time (ms)",
        yaxis_title="Absolute Difference",
        xaxis=dict(range=[0, 400]),
        yaxis=dict(range=[0, diff_data_midi[selected_condition].max() + 5]),
        template="plotly_white"
    )

    return audio_path, fig

@app.callback(
    Output("diff-graph", "figure", allow_duplicate=True),
    [Input("audio-player", "currentTime")],
    [State("condition-dropdown", "value"),
     State("diff-graph", "figure")],
)
def sync_with_audio(current_time, selected_condition, current_fig):
    if current_time is None:
        return current_fig

    # Skaliere die aktuelle Zeit von Sekunden zu Datenzeit (0-400ms über 24s)
    current_data_time = (current_time / 24) * 400

    fig = go.Figure(current_fig)
    fig.update_layout(shapes=[
        dict(
            type="line",
            x0=current_data_time, x1=current_data_time,
            y0=0, y1=1,
            xref="x", yref="paper",
            line=dict(color="red", dash="dot")
        )
    ])
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
import dash
from dash import dcc, html, Input, Output, State
from dash.dependencies import Input, Output

import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objects as go
import os

# Laden des DataFrames diff_data_midi
diff_data_midi = pd.read_csv("/Users/jsenf/Desktop/empra_app/diff_data_midi.csv")

# Liste der verfügbaren Bedingungen und sound_conditions
conditions = [
    "faces_left_faces_right",
    "words_left_words_right",
    "faces_left_words_left",
    "faces_right_words_right",
    "faces_right_words_left",
    "faces_left_words_right"
]
sound_conditions = ["normal", "aesth"]

# Erstellen der Dash-App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("MIDI Note Differences with Audio Synchronization", style={"textAlign": "center"}),

    # Dropdown-Menüs
    html.Div([
        dcc.Dropdown(
            id="condition-dropdown",
            options=[{"label": condition, "value": condition} for condition in conditions],
            value=conditions[0],
            clearable=False,
            style={"width": "48%", "display": "inline-block", "marginRight": "2%"}
        ),
        dcc.Dropdown(
            id="sound-condition-dropdown",
            options=[{"label": sc, "value": sc} for sc in sound_conditions],
            value=sound_conditions[0],
            clearable=False,
            style={"width": "48%", "display": "inline-block"}
        )
    ]),

    # Audio Player
    html.Audio(
        id="audio-player",
        controls=True,
        style={"width": "100%", "marginTop": "20px"}
    ),

    # Graph
    dcc.Graph(id="diff-graph", style={"height": "70vh"}),
])

# Callback: Aktualisiere den Audio-Player und Graphen basierend auf Dropdown-Auswahl und Audio-Zeit
@app.callback(
    [Output("audio-player", "src"),
     Output("diff-graph", "figure")],
    [Input("condition-dropdown", "value"),
     Input("sound-condition-dropdown", "value"),
     Input("audio-player", "currentTime")],
    [State("diff-graph", "figure")]
)
def update_content(selected_condition, selected_sound_condition, current_time, current_fig):
    # Audio-Dateipfad
    audio_path = f"assets/sounds/{selected_condition}_{selected_sound_condition}.wav"

    # Erstellen oder Aktualisieren des Graphen
    if current_time is None:
        # Initialisiere Graph ohne vertikale Linie
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=diff_data_midi["time"],
            y=diff_data_midi[selected_condition],
            mode="lines",
            name=selected_condition,
            line=dict(color="purple")
        ))
        fig.update_layout(
            title=f"Absolute Differences for {selected_condition}",
            xaxis_title="Time (ms)",
            yaxis_title="Absolute Difference",
            xaxis=dict(range=[0, 400]),
            yaxis=dict(range=[0, diff_data_midi[selected_condition].max() + 5]),
            template="plotly_white"
        )
    else:
        # Aktualisiere vorhandenen Graphen mit vertikaler Linie
        current_data_time = (current_time / 24) * 400  # Skaliere Zeit
        fig = go.Figure(current_fig)
        fig.update_layout(shapes=[
            dict(
                type="line",
                x0=current_data_time, x1=current_data_time,
                y0=0, y1=1,
                xref="x", yref="paper",
                line=dict(color="red", dash="dot")
            )
        ])

    return audio_path, fig


if __name__ == "__main__":
    app.run_server(debug=True)
